In [1]:
# initialize project:
using Pkg;
Pkg.activate("./MyProject");

# Find current path
CURRENT = pwd();
LIB_PATH = joinpath(CURRENT, "libs");

# include auxiliary Julia scripts:
for file in readdir(LIB_PATH)
    # Check if the file ends with ".jl" extension
    if endswith(file, ".jl")
        include(joinpath(LIB_PATH, file))
    end
end

using Polymake
const pm = Polymake;
const g = GAP.Globals;

  Activating project at `~/Documents/GitHub/StabTheory/MyProject`


In [2]:
i3 = canonical_maximal_isotropic(3)
symplectic_orbit = SympOrbit(3, SympPerm(3), Set(i3)).Orbit

I3loc = local_isotropics(symplectic_orbit, 3)
A3loc = stabilizer_coefficients(3, I3loc)

216×64 Matrix{Int64}:
 1   1  0   0   0   0  0   0   1   1  …   0  0  0   1   1  0  0  0  0  0  0
 1  -1  0   0   0   0  0   0  -1   1      0  0  0   1  -1  0  0  0  0  0  0
 1  -1  0   0   0   0  0   0   1  -1      0  0  0   1  -1  0  0  0  0  0  0
 1   1  0   0   0   0  0   0  -1  -1      0  0  0   1   1  0  0  0  0  0  0
 1  -1  0   0   0   0  0   0   1  -1      0  0  0  -1   1  0  0  0  0  0  0
 1   1  0   0   0   0  0   0  -1  -1  …   0  0  0  -1  -1  0  0  0  0  0  0
 1   1  0   0   0   0  0   0   1   1      0  0  0  -1  -1  0  0  0  0  0  0
 1  -1  0   0   0   0  0   0  -1   1      0  0  0  -1   1  0  0  0  0  0  0
 1   0  0   1   1   0  0   1   0   0      0  0  0   0   0  0  0  0  0  0  0
 1   0  0  -1   1   0  0  -1   0   0      0  0  0   0   0  0  0  0  0  0  0
 ⋮                  ⋮                 ⋱         ⋮                ⋮        
 1   0  0   1   0   0  0   0  -1   0      0  0  0   0   0  0  0  0  0  0  0
 1   1  0   0   1   1  0   0   0   0      1  0  0   0   0  0  0  0 

In [3]:
all_locals = Set{Vector{Int}}(
    [
        IIX, IIY, IIZ, IXI, IYI, IZI, XII, YII, ZII
    ]
)

E3 = find_local_closure(all_locals)
det_vertices = find_all_possible_local_value_assignments(E3)
det_vertices_matrix = hcat([value_assignment_to_pauli_basis(v, 3) for v in det_vertices]...)

64×512 Matrix{Int64}:
  1   1   1   1   1   1   1   1   1  …   1   1   1   1   1   1   1   1   1
  1   1  -1  -1  -1   1  -1   1  -1      1  -1   1   1  -1   1   1  -1   1
  1   1  -1   1  -1  -1   1   1   1     -1   1  -1   1   1  -1   1  -1   1
 -1   1  -1   1   1  -1   1   1   1      1   1   1   1  -1  -1   1  -1  -1
 -1  -1   1  -1   1  -1   1   1  -1      1   1   1  -1   1   1   1   1  -1
 -1  -1  -1   1  -1  -1  -1   1   1  …   1  -1   1  -1  -1   1   1  -1  -1
 -1  -1  -1  -1  -1   1   1   1  -1     -1   1  -1  -1   1  -1   1  -1  -1
  1  -1  -1  -1   1   1   1   1  -1      1   1   1  -1  -1  -1   1  -1   1
  1   1  -1   1  -1   1   1  -1   1      1  -1  -1   1   1   1   1  -1   1
  1   1   1  -1   1   1  -1  -1  -1      1   1  -1   1  -1   1   1   1   1
  ⋮                   ⋮              ⋱           ⋮                   ⋮  
 -1  -1  -1   1   1   1  -1  -1   1  …   1  -1   1   1   1   1  -1  -1   1
 -1   1  -1  -1  -1   1  -1   1  -1      1   1  -1  -1  -1  -1  -1  -1   1
 -1  

In [4]:
find_rank_count_for_given_given_value_assignment(first(det_vertices), 3, A3loc)

63

In [5]:
ex = Vector{Float64}(value_assignment_to_pauli_basis(first(det_vertices), 3))

64-element Vector{Float64}:
  1.0
  1.0
  1.0
 -1.0
 -1.0
 -1.0
 -1.0
  1.0
  1.0
  1.0
  ⋮
 -1.0
 -1.0
 -1.0
 -1.0
  1.0
 -1.0
 -1.0
 -1.0
  1.0

In [100]:
all_cncs = generate_all_cncs(3, [0,3])
failed_extended_cncs = Set{MaximalCnc}()
vertex_count = 0
e3_decomp_count = 0
extension_not_possible_count = 0
no_vertices_found_count = 0
extended_vertices_count = 0
for cnc in all_cncs
    rank = find_rank_count_for_given_given_value_assignment(cnc.value_assignment, 3, A3loc)
    if rank != 63
        val_asg = cnc.value_assignment
        cnc_pauli = Vector{Float64}(value_assignment_to_pauli_basis(val_asg, 3))
        solver = Initializer(3, cnc_pauli, det_vertices_matrix)

        if !solver.Feasible
            fillable_w3s = get_fillable_weight_3s(find_full_set_for_given_cnc_set(cnc.cnc_set))
            
            num_indep = length(fillable_w3s)

            if num_indep == 0
                extension_not_possible_count += 1
                push!(failed_extended_cncs, cnc)
                continue
            end

            value_assignments = Vector{Dict{Vector{Int},Int}}()
            loop_count = 0
            inverse_vertex_pair_found = false
            while length(value_assignments) < 4 && loop_count < 2^10
                value_array = rand(0:1, num_indep)

                value_assignment = copy(val_asg)
                inv_value_assignment = copy(val_asg)

                for (p, v) in zip(fillable_w3s, value_array)
                    value_assignment[p] = (-1)^v
                    inv_value_assignment[p] = (-1)^(v + 1)
                end
                rank = find_rank_count_for_given_given_value_assignment(value_assignment, 3, A3loc)
                inv_rank = find_rank_count_for_given_given_value_assignment(inv_value_assignment, 3, A3loc)

                if rank == 63 && inv_rank == 63
                    inverse_vertex_pair_found = true
                    break
                elseif rank == 63
                    push!(value_assignments, value_assignment)
                elseif inv_rank == 63                
                    push!(value_assignments, inv_value_assignment)
                end

                loop_count += 1
            end

            if inverse_vertex_pair_found
                extended_vertices_count += 1
                count += 1
                continue
            end

            if length(value_assignments) == 0
                no_vertices_found_count += 1
                push!(failed_extended_cncs, cnc)
                continue
            end
            extended_vertices = hcat([value_assignment_to_pauli_basis(v, 3) for v in value_assignments]...)
            solver2 = Initializer(3, cnc_pauli, extended_vertices)
            if solver2.Feasible
                extended_vertices_count += 1
            else
                push!(failed_extended_cncs, cnc)
            end
        else
            e3_decomp_count += 1
        end 
    else
        vertex_count += 1  
    end
end

println("finished")

finished


In [101]:
vertex_count, e3_decomp_count, extended_vertices_count, no_vertices_found_count, length(failed_extended_cncs)

(0, 9864, 28080, 0, 0)